In [75]:
import numpy as np
import codecs
from csv import DictReader
import pandas as pd

##Iskanje pravilnega enkodiranja -> poskusi vse
encoding_list = [
    '646', 'ansi_x3.4-1968', 'ansi_x3.4-1986', 'ascii', 'big5', 'big5hkscs', 'cp037', 'cp1026', 'cp1125', 'cp1140', 
    'cp1250', 'cp1251', 'cp1252', 'cp1253', 'cp1254', 'cp1255', 'cp1256', 'cp1257', 'cp1258', 'cp273', 'cp278', 'cp280', 
    'cp284', 'cp285', 'cp297', 'cp437', 'cp500', 'cp720', 'cp737', 'cp775', 'cp850', 'cp852', 'cp855', 'cp856', 'cp857', 
    'cp858', 'cp860', 'cp861', 'cp862', 'cp863', 'cp864', 'cp865', 'cp866', 'cp869', 'cp874', 'cp875', 'cp932', 'cp949', 
    'cp950', 'cp1006', 'cp1025', 'cp1252', 'cp437', 'cp500', 'cp720', 'cp737', 'cp775', 'cp850', 'cp852', 'cp855', 'cp856', 
    'cp857', 'cp858', 'cp860', 'cp861', 'cp862', 'cp863', 'cp864', 'cp865', 'cp866', 'cp869', 'cp874', 'cp875', 'cp932', 
    'cp949', 'cp950', 'euc_jis_2004', 'euc_jisx0213', 'euc_jp', 'euc_kr', 'gb2312', 'gb18030', 'gbk', 'hz', 'iso2022_jp', 
    'iso2022_jp_1', 'iso2022_jp_2', 'iso2022_jp_2004', 'iso2022_jp_3', 'iso2022_jp_ext', 'iso2022_kr', 'iso8859_1', 
    'iso8859_10', 'iso8859_11', 'iso8859_13', 'iso8859_14', 'iso8859_15', 'iso8859_16', 'iso8859_2', 'iso8859_3', 'iso8859_4', 
    'iso8859_5', 'iso8859_6', 'iso8859_7', 'iso8859_8', 'iso8859_9', 'johab', 'koi8_r', 'koi8_t', 'koi8_u', 'kz1048', 
    'latin_1', 'mac_cyrillic', 'mac_greek', 'mac_iceland', 'mac_latin2', 'mac_roman', 'mac_turkish', 'ptcp154', 'shift_jis', 
    'shift_jis_2004', 'shift_jisx0213', 'utf_32', 'utf_32_be', 'utf_32_le', 'utf_16', 'utf_16_be', 'utf_16_le', 'utf_7', 
    'utf_8', 'utf_8_sig'
]
a = []
for encoding in encoding_list:
    worked = True
    try:
        data = pd.read_csv('data/Nio_vozila_1.csv', sep=';', encoding=encoding, on_bad_lines='skip', low_memory=False, nrows=50)
    except:
        worked = False
    if worked:
        try:
            print(encoding,data.at[39,"Izvajalna enota prve registracije"], end="")
        except:
            print(encoding, end="")


data = pd.read_csv('data/Nio_vozila_1.csv', sep=';', encoding="iso8859_2", on_bad_lines='skip', low_memory=False)

['КTAJERSKI AVTO DOM D.O.O.', 'ЉTAJERSKI AVTO DOM D.O.O.', 'ٹTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'ΛTAJERSKI AVTO DOM D.O.O.', 'ŖTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'ŐTAJERSKI AVTO DOM D.O.O.', 'іTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'ךTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', '├TAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'КTAJERSKI AVTO DOM D.O.O.', '¦TAJERSKI AVTO DOM D.O.O.', '\x8aTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'ΛTAJERSKI AVTO DOM D.O.O.', 'ŖTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'ŐTAJERSKI AVTO DOM D.O.O.', 'іTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', 'ךTAJERSKI AVTO DOM D.O.O.', 'èTAJERSKI AVTO DOM D.O.O.', '├TAJERSKI

In [66]:
cars = data[data['J-Kategorija in vrsta vozila (opis)'] == 'osebni avtomobil']
cars

,B-Datum prve registracije vozila,2A-Datum prve registracije vozila v SLO,1K-Oznaka potrdila o skladnosti,Datum statusa vozila,Status vozila (id),Status vozila (opis),Izvajalna enota prve registracije,4A-Registrsko obmocje tablice prve registracije,4A-Podvrsta tablice prve registracije,C-Starost uporabnika vozila,...,Y3-Visina,Z1-Dovoljene pnevmatike in platisca,Z2-Homologacijska oznaka vlecne naprave,Z21-D Ä vrednost,Datum izdaje potrdila o skladnosti,Okoljevarstvena oznaka,Pogonska os z zracnim vzmetenjem ali priznanim ekvivalentom,8B-Namen vozila,Komerc oznaka do prvega /,Prevozeni kilometri milje
0,12.04.2017,12.04.2017,SA,12.04.2017,1,registrirano,TAJERSKI AVTO DOM D.O.O.,CELJE,NT,70.0,...,1441.0,"205/55 R16 (91) H, 6.5 J 16 41205/50 R17 (93) ...",NaN,NaN,11.04.2017,EURO 6,NaN,NaN,MEGANE,42501.0
2,28.10.2011,28.10.2011,SA,07.05.2021,1,registrirano,TAJERSKI AVTO DOM D.O.O.,MARIBOR,NT,27.0,...,1488.0,"185/70R14 88T, 185/65R15 88T, 195/60R15 88T, 1...",E26 55R 01 032*00,"6,9",27.10.2011,EURO 5,NaN,NaN,CORSA,99818.0
7,23.04.2020,23.04.2020,SA,23.04.2020,1,registrirano,UPRAVNA ENOTA ALEC,LJUBLJANA,NT,56.0,...,1671.0,"235/65R17 101H, 235/60R18,101H, 235/55R19 101W...",NaN,NaN,09.03.2020,EURO 6,NaN,NaN,CR-V,NaN
8,25.04.2016,25.04.2016,SA,25.04.2017,1,registrirano,AMZS d.d. PE Ptuj,MARIBOR,NT,57.0,...,1495.0,"185/65R15 88T, 185/60R16 86H",NaN,NaN,22.04.2016,EURO 6,NaN,NaN,MAZDA 2,40014.0
10,18.09.2018,18.09.2018,SA,18.09.2018,1,registrirano,AMZS D.D. POSLOVALNICA LENART,MURSKA SOBOTA,NT,48.0,...,1474.0,"185/65 R15 (88)T 6.0J x 15, T125/80 D15 (95...",NaN,NaN,17.09.2018,EURO 6,NaN,NaN,i20,46402.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649995,04.01.2011,09.03.2015,SB,16.04.2018,1,registrirano,AVTO KRKA CE d.o.o. PE ENTJUR,CELJE,NT,52.0,...,1467.0,"235/50 R19 103H M+S, 255/45 R19 104Y, 265/40 R...",e13 00 3161,"12,60",09.03.2015,EURO 5,NaN,NaN,A8,256504.0
649996,28.05.2020,28.05.2020,SA,28.05.2020,1,registrirano,"REGIA GROUP d.d., PE LABORE",LJUBLJANA,NT,56.0,...,1648.0,"195/60 R16 83T 6.50J16 (e=20), 195/65 R15 83T ...",NaN,NaN,25.05.2020,EURO 6,NaN,NaN,C3 AIRCROSS,NaN
649997,13.05.2011,16.10.2015,SB,11.07.2022,1,registrirano,"M CENTER d.o.o., PE Ravne na Korokem",MARIBOR,NT,24.0,...,1538.0,"185/65R15 (88T), 195/55R16 (87T)",NaN,NaN,02.07.2015,EURO 5,NaN,NaN,C3,242646.0
649998,30.05.2008,30.05.2008,SA,30.05.2008,1,registrirano,AVTO KRKA NM d.o.o.PE NOVO MESTO - ukinjen,NOVO MESTO,TN,43.0,...,1498.0,"195/65R1591V,205/55R1691V,225/45R1791V",NaN,NaN,16.05.2008,EURO 4,NaN,NaN,BRAVO,266113.0


### Ugotovitev 
Če poskusimo odpreti datoteke z katerikolim enkodiranjem, dobimo izpis napačen -> napaka v datoteki?